In [1]:
import pint
ureg = pint.UnitRegistry()

import numpy as np
import numpy.linalg as la

import scipy as sp

import matplotlib.pyplot as plt

from fluids.atmosphere import ATMOSPHERE_1976 as atm

In [2]:
# utilities
def si(v):
    """Helper function to get si units magnitude on variables using pint"""

    if isinstance(v, type(1*ureg.m)):
        return v.to_base_units().magnitude
    elif isinstance(v, Quaternion):
        return v.q
    else:
        return v
    
def quat_to_rot(qq):
    
    """Convert a (np.array) quaternion to a rotation matrix"""
    
    qq  = qq[:,0]
    
    R = np.array([[ 1 - 2*qq[2]**2 - 2*qq[3]**2,  2*qq[1]*qq[2] - 2*qq[0]*qq[3],    2*qq[1]*qq[3] + 2*qq[0]*qq[2]],
                     [2*qq[1]*qq[2] + 2*qq[0]*qq[3],    1 - 2*qq[1]**2 - 2*qq[3]**2,    2*qq[2]*qq[3] - 2*qq[0]*qq[1]],
                     [2*qq[1]*qq[3] - 2*qq[0]*qq[2],  2*qq[2]*qq[3] + 2*qq[0]*qq[1],     1 - 2*qq[1]**2 - 2*qq[2]**2]])

    return R
    
def unit_vector(v):
    """Return a unit vector in the direction of v."""
    return v/la.norm(v)

def angle_between(va, vb):
    
    """ Return the angle between two column vectors using the dot product"""
    
    th = np.arccos(np.dot(unit_vector(va.T), unit_vector(vb)))
    
    return float(th)
    

In [3]:
class Quaternion():

    """Note, does not support Pint units """
    
    def __init__(self, s, vx, vy, vz):
        """ Creates a quaternion. Expects floats"""
        self.q = np.array([[s, vx, vy, vz]]).T
    
    @classmethod
    def from_angle(cls, theta, axis):
        """A rotation of angle `theta` about the axis `ax, ay, az`. Allows the axis to be not normalized"""
        
        axis = unit_vector(axis)
        
        s = np.cos(theta/2)
        v = np.sin(theta/2)*axis        

        return cls(s, *v)
    
        
    def rot_matrix(self, qq=None):
        
        if qq is None:
            qq = self.q
            
        return quat_to_rot(qq)
    
    
    def __repr__(self):
        return str(self.q)
        
    

In [4]:
q = Quaternion.from_angle(6.0*np.pi/180.0, np.array([0,1,0]))
q

[[0.99862953]
 [0.        ]
 [0.05233596]
 [0.        ]]

In [57]:
class Rocket():
    
    def __init__(self):
    
        self.T_max = 1000*ureg.N
        
        self.M_0 = 40*ureg.kg # wet mass of rocket
        self.M_d = 20*ureg.kg # dry mass of rocket
        
        self.I_normal = 50*ureg.kg*ureg.m**2
        self.I_axial  = 0.05*ureg.kg*ureg.m**2
        
        self.ref_length = 4.0*ureg.m
        self.diameter = 6*ureg.inch
        
        self.fin_height = 6*ureg.inch
        self.fin_count = 4
        self.fin_cant = 0.05*ureg.rad
        
        pass
    
    def get_thrust(self, t, state):
        """Return the thrust at current time using current time and state vector, used for simulator."""
        
        if t < 10*u.s:
            return self.T_max
        else:
            return 0*ureg.N
        
    def rate_I_axial(self, t, state):
        """Rate of change of """
        pass
    
    def get_xcp(self, alpha, Re, Mach):
        
        return 2.5*ureg.m
    
    def get_xcm(self, mass):
        
        return 2.0*ureg.m
    
    def get_fin_locations(self):
        
        th0 = 0*ureg.degree
        dth = 2*np.pi*ureg.rad / self.fin_count
        r = self.diameter/2 + self.fin_height/2
        
        for i in range(self.fin_count):
            th = th0+i*dth
            
            x=r*np.cos(th)
            y=r*np.sin(th)
            z=self.ref_length
            yield np.array([si(xx) for xx in [x,y,z]])*ureg.m
            
    def get_fin_axes(self, roll_axis_in_earth):
        
        for loc in self.get_fin_locations(): #gets me the x,y,z. I only need the x,y components
        
            fin_direction = unit_vector(np.array([[si(loc[0]), si(loc[1]), 0.0]]).T)
            
            fin_normal = np.cross(fin_direction, roll_axis_in_earth, axis=0)
            
            fin_quat = Quaternion.from_angle(self.fin_cant, fin_axis)
            
            
            
            
        
        
        
        
    
class Fin():
    
    def __init__(self, root_chord=12*ureg.inch, tip_chord=6*ureg.inch, height=6*ureg.inch, leading_edge_sweep=5*ureg.inch, loc_of_trailing_egde_root=[0*ureg.inch, 0*ureg.degree]):
        
        self.root_chord=root_chord
        self.tip_chord=tip_chord
        self.height=height
        self.leading_edge_sweep=leading_edge_sweep
        self.loc_of_trailing_egde_root=loc_of_trailing_egde_root
        
    def get_cp_location(self, body_diameter=6*ureg.inch):
        # return body axis loction of cp
        
        #todo: implement correctly
        
        r = body_diameter/2 + self.height/2
        z = 0
        th=self.loc_of_trailing_egde_root[-1]
        
        return [r,z,th]
        
        
        

In [58]:
rocket = Rocket()

In [65]:
[-si(f)+[0, 0, si(rocket.get_xcm(5))] for f in list(rocket.get_fin_locations())]

[array([-0.1524,  0.    , -2.    ]),
 array([-9.33180861e-18, -1.52400000e-01, -2.00000000e+00]),
 array([ 1.52400000e-01, -1.86636172e-17, -2.00000000e+00]),
 array([ 2.79954258e-17,  1.52400000e-01, -2.00000000e+00])]

In [60]:
[0, 0, si(rocket.get_xcm(5))]*ureg.m

array([0., 0., 2.]) <Unit('meter')>

In [48]:
rocket.get_xcm(5)

2.0 <Unit('meter')>

In [86]:
for loc in rocket.get_fin_locations():
    print(si(loc[0]))

0.15239999999999998
9.331808609502831e-18
-0.15239999999999998
-2.7995425828508496e-17


In [78]:
class Environment():
    
    def __init__(self):
        
        # define constants
        self.gamma = 1.4 * ureg("")
        self.R = 287*ureg.J/ureg.kg/ureg.degK
        self.ref_temp = 291.15*ureg.degK
        self.ref_visc = 1.827e-7*ureg.Pa/ureg.s
        self.M_earth = 5.974e24*ureg.kg
        self.R_earth = 6378100*ureg.m
        self.G = 6.673e-11*ureg.m**3/ureg.kg/ureg.s
        self.g_0 = 9.81*ureg.m/ureg.s**2
        self.rho_0 = 1.225*ureg.kg/ureg.m**3
           
    def get_rho(self, z):
        
        try:
            return atm(z).rho #return in si
        except:
            return atm((z.to(ureg.m)).magnitude).rho*ureg.kg/ureg.m**3
    
    def get_mu(self, z):
        
        try:
            return atm(z).mu #return in si
        except:
            return atm((z.to(ureg.m)).magnitude).mu*ureg.Pa*ureg.s

    def get_temp(self, z):
        
        try:
            return atm(z).T #return in si
        except:
            return atm((z.to(ureg.m)).magnitude).T*ureg.degK
    
    def get_speed_of_sound(self, z):
        
        try:
            return atm(z).v_sonic#return in si
        except:
            return atm((z.to(ureg.m)).magnitude).v_sonic*ureg.m/ureg.s
        
    def frozen_wind(t, state):
        
        # function to describe the wind vector (in earth frame) as a function of time or state
        
        return np.array([[1,0,0]]).T

    

In [79]:
env = Environment()

In [9]:
class State():
    
    def __init__(self, pos, quaternion, lin_mom, ang_mom, mass, inertia):
        
        self.pos = pos
        self.quat = quaternion
        self.lin_mom = lin_mom
        self.ang_mom = ang_mom
        self.mass = mass
        self.inertia = inertia
        
        self.inertia_matrix = self.get_inertia_matrix(inertia)
        
        self.state = [self.pos, self.quat, self.lin_mom, self.ang_mom, self.mass, self.inertia]
        
        return
    
    def get_state_vector(self):
        
        return np.vstack([si(s) for s in self.state])
    
    def get_inertia_matrix(self, inertia):
        """Convert I_axial, I_normal to inertia matrix"""
        if inertia is None:
            inertia = self.inertia
            
        I_normal = inertia[0][0]
        I_axial  = inertia[1][0]
        inertia_matrix =  np.diagflat([si(I_normal), si(I_normal), si(I_axial)])*ureg.kg*ureg.m**2
            
        return inertia_matrix
    
    def __repr__(self):
        
        return repr(self.state)
        
    

In [10]:
pos = np.array([[0,0,0]]).T*ureg.m
#quat = q
lin_mom = np.array([[1,0,0]]).T*ureg.kg*ureg.m/ureg.s
ang_mom = np.array([[0,70,1000]]).T*ureg.kg*ureg.m**2/ureg.s
mass = 40*ureg.kg
inertia = np.array([[50,0.01]]).T*ureg.kg*ureg.m**2
state = State(pos, q, lin_mom, ang_mom, mass, inertia)

In [11]:
state.get_state_vector()

array([[0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [9.98629535e-01],
       [0.00000000e+00],
       [5.23359562e-02],
       [0.00000000e+00],
       [1.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       [7.00000000e+01],
       [1.00000000e+03],
       [4.00000000e+01],
       [5.00000000e+01],
       [1.00000000e-02]])

In [12]:
state.inertia_matrix

array([[5.e+01, 0.e+00, 0.e+00],
       [0.e+00, 5.e+01, 0.e+00],
       [0.e+00, 0.e+00, 1.e-02]]) <Unit('kilogram * meter ** 2')>

In [74]:
class Simulation():
    
    def __init__(self, rocket, environment):
        
        self.rocket = rocket
        self.env = environment
        
        self.ref_axis_Y = np.array([[1,0,0]]).T # yaw axis
        self.ref_axis_P = np.array([[0,1,0]]).T # pitch axis
        self.ref_axis_R = np.array([[0,0,1]]).T # roll axis
        
        return
        
    def set_initial_state(self, state):
        #state of the rocket is described by 
        # position (x,y,z), orientation quaternion (s, vx, vy, vz), Linear momentum (Px, Py, Pz), angular Momentum (Lx, Ly, Lz), mass (M), inertias (I_axial, I_normal)
        
        self.state = state
        self.initial_state = state
        
        return
    
    def dynamics(self, t, state):
        
        # take in time, state (assuming SI), return the rate of change of states in si. the noise and wind is added internally.
        
        
        pos, quat, R, lin_mom, ang_mom, mass, inertia_matrix = self.extract_states(state)
        RA = R @ self.ref_axis_R # roll axis
        
        # calculate rate of change of position: linear velocity
        lin_vel = lin_mom/mass

        # note, R is from earth to body
        # calculate rate of change of quaternion
        ang_vel = R @ la.inv(inertia_matrix) @ R.T @ ang_mom
        
        # calculate quaternion derivative
        qdot = self.get_qdot(quat, lin_vel, ang_vel)
        
        # compute wind
        wind = env.frozen_wind(t, state)
        
        # calculate angle of attack, re, mach, vcm, vinfty using V_cm
        
        V_cm = lin_vel + wind
        
        
        # calculate all these parameters, automatically performing convergence as necessary
        alpha, re, mach, xbar, V_omega, V_infty = self.calc_parameters_converged(V_cm, R, RA, pos, mass, ang_vel)
        
        # calculate fins
        #get fin locations
        fin_loc_relative = [-si(f)+[0, 0, si(rocket.get_xcm(5))] for f in list(rocket.get_fin_locations())]
        # apply rotation
        fin_arm = [-R@fin_loc for fin_loc in fin_loc_relative]
        
        #velocity at fin:
        V_fin = [V_cm + np.cross(arm, ang_vel, axis=0) for arm in fin_arm]
        
        #
        
        
        
        
        
        # calculate forces:
        F_thrust = R.T @ self.get_thrust_force(t, state)
        
        F_gravity = mass * self.get_gravitational_accel(pos)
        
        # calculate qinfty
        z = state[3]
        q_infty = 0.5*env.get_rho(z)*V_infty.dot(V_infty)
        
        F_axial = 0 #TODO: Define the function here

        total_force = F_thrust + F_gravity# + F_axial + F_normal
        
        
        #calculate total 
        
        

    def calc_parameters_converged(self, V_cm, R, RA, pos, mass, ang_vel):
        if la.norm(ang_vel) < 0.1:
            
            return self.calc_parameters(V_cm, V_cm, R, RA, pos, mass, ang_vel)

        else:
            #calculate using V_cm
            alpha_old, re, mach, xbar, V_omega, V_infty = self.calc_parameters(V_cm, V_cm, R, RA, pos, mass, ang_vel)

            # calculate new alpha
            alpha, re, mach, xbar, V_omega, V_infty = self.calc_parameters(V_infty, V_cm, R, RA, pos, mass, ang_vel)


            # if the difference is large, re-estimate
            while abs(alpha_old - alpha) > 1e-6:

                alpha_old = alpha
                alpha, re, mach, xbar, V_omega, V_infty = self.calc_parameters(V_infty, V_cm, R, RA, pos, mass, ang_vel)


        return alpha, re, mach, xbar, V_omega, V_infty 
        
    def calc_parameters(self, V_infty, V_cm, R, RA, pos, mass, ang_vel):
        # note, V_infty here is a guess, but the corrected value comes out of the function
            
        alpha = self.calc_alpha(V_infty, RA) #using V_cm as an approximation for V_infty
        
        z = pos[2][0]
        re = self.calc_Re(la.norm(V_infty), z)
        mach = self.calc_Mach(la.norm(V_infty), z)

        xbar = abs(self.rocket.get_xcp(alpha, re, mach) - self.rocket.get_xcm(mass)).to(ureg.m).magnitude
        
        V_omega = xbar * np.sin(angle_between(RA, ang_vel)) * np.cross(RA, ang_vel, axis=0)# linear speed at the center of pressure due to the rotation rate of the rocket

        V_infty = V_cm + V_omega

        return alpha, re, mach, xbar, V_omega, V_infty

    def calc_Re(self, Vinf, z):
        """Calculate reynolds number for a given vinf and altitude z"""
        
        rho = self.env.get_rho(z)
        mu  = self.env.get_mu(z)
        
        X = (self.rocket.ref_length.to(ureg.m)).magnitude #using the ref length used for coefficient calculation
        
        Re = rho*Vinf*X/mu
        
        return Re
        
        
    def calc_Mach(self, Vinf, z):
        
        a = self.env.get_speed_of_sound(z)
        
        return Vinf/a
    
    def calc_alpha(self, Vinf, RA):
        """Calc alpha for a given wind speed Vinf and roll axis RA"""
        
        return angle_between(Vinf, RA)
        
        

    def get_gravitational_accel(self, pos):
        
        # todo: possibly include earth radius
        g = 9.81
        
        return np.array([[0,0,-g]]).T
                              
        
    def get_thrust_force(self, t, state):
        
        """Wrapper for providing thrust. Must return vector of thrust direction in body axis"""
        
        if t<10:
            thrust = si(self.rocket.T_max)
        else:
            thrust = 0
            
        # convert to axis
        roll_axis = np.array([[0,0,1]]).T
        
        
        return thrust*roll_axis
        
        
        
    def extract_states(self, state):
        
        pos = state[0:3]
        quat = state[3:7]
        R = self.rotation_matrix(quat)
        lin_mom = state[7:10]
        ang_mom = state[10:13]
        mass = state[13]
        inertia = state[14:16]
        
        inertia_matrix = self.inertia_matrix(*inertia)
        
        return pos, quat, R, lin_mom, ang_mom, mass, inertia_matrix
        
   
    def get_qdot(self, quat, lin_vel, ang_vel):

        """Compute the rate of change of the quaternion"""
        s = quat[0][0]
        sdot = 0.5 * float( np.dot(ang_vel.T, lin_vel))
        vdot = 0.5 * s*ang_vel + np.cross(ang_vel, ang_vel,axis=0)
        qdot =  np.vstack([sdot, vdot])
        return qdot
        
        
    
    def rotation_matrix(self,qq):
        
        qq = qq[:,0]
        
        """Return the rotation matrix associated with a quaternion qq"""
        R = np.array([[1 - 2*qq[2]**2 - 2*qq[3]**2,   2*qq[1]*qq[2] - 2*qq[0]*qq[3],    2*qq[1]*qq[3] + 2*qq[0]*qq[2]],
                     [2*qq[1]*qq[2] + 2*qq[0]*qq[3],    1 - 2*qq[1]**2 - 2*qq[3]**2,    2*qq[2]*qq[3] - 2*qq[0]*qq[1]],
                     [2*qq[1]*qq[3] - 2*qq[0]*qq[2],  2*qq[2]*qq[3] + 2*qq[0]*qq[1],     1 - 2*qq[1]**2 - 2*qq[2]**2]])

        return R
    
    def inertia_matrix(self, I_normal, I_axial):
        """Convert I_normal, I_axial to 3x3 inertia matrix"""
        
        return np.diagflat([I_normal, I_normal, I_axial])


        

In [75]:
sim = Simulation(rocket, env)

In [76]:
state.get_state_vector();

In [77]:
ans = sim.dynamics(0, state.get_state_vector())

[array([[-1.73735911e+00],
       [-1.51606948e+04],
       [ 5.04870889e-01]]), array([[ 1.50717584e+04],
       [-4.18113853e+00],
       [-1.58367755e+03]]), array([[-1.78196350e+00],
       [ 1.51523325e+04],
       [ 8.04885056e-02]]), array([[-1.50752777e+04],
       [-4.18113853e+00],
       [ 1.58426291e+03]])]


In [17]:
ans